Import needed modules

In [21]:
import pandas as pd
import os
import subprocess

Import data

In [2]:
# read the data
TP_data = pd.read_csv("./data/TP_SIDERs_coor.csv", sep=",", header=0)
Software_data = pd.read_csv("./data/Results_Program_29-05-54.csv", sep=",", header=0)
print(f"'True Positives' row length: {TP_data.shape[0]}\n",
      f"Software row length: {Software_data.shape[0]}")

'True Positives' row length: 1606
 Software row length: 5972


Check how many elements for each chromosome in the "True Positive" data

In [10]:
TP_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01      7
LinJ.02     18
LinJ.03      8
LinJ.04     18
LinJ.05     18
LinJ.06     22
LinJ.07     27
LinJ.08     18
LinJ.09     21
LinJ.10     26
LinJ.11     39
LinJ.12     36
LinJ.13     22
LinJ.14     66
LinJ.15     25
LinJ.16     34
LinJ.17     34
LinJ.18     30
LinJ.19     40
LinJ.20     37
LinJ.21     39
LinJ.22     27
LinJ.23     60
LinJ.24     57
LinJ.25     42
LinJ.26     45
LinJ.27     63
LinJ.28     46
LinJ.29     74
LinJ.30     71
LinJ.31     80
LinJ.32     69
LinJ.33     85
LinJ.34     96
LinJ.35    110
LinJ.36     96
Name: count, dtype: int64

Check how many elements for each chromosome in the software data

In [11]:
Software_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     50
LinJ.02     61
LinJ.03     43
LinJ.04     67
LinJ.05     78
LinJ.06     72
LinJ.07     91
LinJ.08    163
LinJ.09     89
LinJ.10    117
LinJ.11    107
LinJ.12    151
LinJ.13     95
LinJ.14    118
LinJ.15     83
LinJ.16    119
LinJ.17    121
LinJ.18    135
LinJ.19    164
LinJ.20    122
LinJ.21    121
LinJ.22    147
LinJ.23    174
LinJ.24    146
LinJ.25    123
LinJ.26    150
LinJ.27    212
LinJ.28    163
LinJ.29    326
LinJ.30    234
LinJ.31    332
LinJ.32    236
LinJ.33    270
LinJ.34    517
LinJ.35    367
LinJ.36    408
Name: count, dtype: int64

Difference in numbers in the software data compared to the "True Positives"

In [12]:
Software_data["sseqid"].value_counts(sort=False) - TP_data["sseqid"].value_counts(sort=False)

sseqid
LinJ.01     43
LinJ.02     43
LinJ.03     35
LinJ.04     49
LinJ.05     60
LinJ.06     50
LinJ.07     64
LinJ.08    145
LinJ.09     68
LinJ.10     91
LinJ.11     68
LinJ.12    115
LinJ.13     73
LinJ.14     52
LinJ.15     58
LinJ.16     85
LinJ.17     87
LinJ.18    105
LinJ.19    124
LinJ.20     85
LinJ.21     82
LinJ.22    120
LinJ.23    114
LinJ.24     89
LinJ.25     81
LinJ.26    105
LinJ.27    149
LinJ.28    117
LinJ.29    252
LinJ.30    163
LinJ.31    252
LinJ.32    167
LinJ.33    185
LinJ.34    421
LinJ.35    257
LinJ.36    312
Name: count, dtype: int64

Let's group data by chromosomes

In [13]:
TP_data_grouped = TP_data.groupby("sseqid")
Software_data_grouped = Software_data.groupby("sseqid")

Let's make the main code

In [18]:
# Check actual path
os.getcwd()

'/home/rfpacheco/Documents/Work_CBMSO/Testing_Leishmania_project/12.Sensibility_script'

In [19]:
# Folder path
bedops_path = "./bedops_data"

First option with different strands:

In [22]:
def columns_to_numeric(data_input, columns_to_convert = []):
    for column in columns_to_convert:
        data_input[column] = pd.to_numeric(data_input[column], errors='coerce')
    return data_input

In [26]:
for (name1, group1), (name2, group2) in zip(TP_data_grouped, Software_data_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(bedops_path, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_plus, group1_minus = group1[group1["sstrand"] == "plus"].copy(), group1[group1["sstrand"] == "minus"].copy()
    group2_plus, group2_minus = group2[group2["sstrand"] == "plus"].copy(), group2[group2["sstrand"] == "minus"].copy()
    # ------------------------------------------------------------------------------
    group1_plus = group1_plus[["sseqid", "sstart", "send"]]
    group1_minus = group1_minus[["sseqid", "sstart", "send"]]
    group2_plus = group2_plus[["sseqid", "sstart", "send"]]
    group2_minus = group2_minus[["sseqid", "sstart", "send"]]
    # ------------------------------------------------------------------------------
    group1_minus[["sstart", "send"]] = group1_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    group2_minus[["sstart", "send"]] = group2_minus[["send", "sstart"]]  # because minus strand coordinates are reversed
    # ------------------------------------------------------------------------------
    path_group1_plus = os.path.join(path_chr, "group1_plus.bed")
    path_group1_minus = os.path.join(path_chr, "group1_minus.bed")
    path_group2_plus = os.path.join(path_chr, "group2_plus.bed")
    path_group2_minus = os.path.join(path_chr, "group2_minus.bed")
    # ------------------------------------------------------------------------------
    group1_plus.to_csv(path_group1_plus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group1_minus.to_csv(path_group1_minus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_plus.to_csv(path_group2_plus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_minus.to_csv(path_group2_minus, sep="\t", header=False, index=False)  # tabular sep because of bed format
    # ------------------------------------------------------------------------------
    path_group1_list = [path_group1_plus, path_group1_minus]
    path_group2_list = [path_group2_plus, path_group2_minus]
    TP_df_list = [group1_plus, group1_minus]
    # ------------------------------------------------------------------------------
    comparison_dict = {}
    for index, (path1, path2, TP_group) in enumerate(zip(path_group1_list, path_group2_list, TP_df_list)):  # There are the paths to the files
        condiction = 1
        # print(f"group1:  {group1}\n"
        #       f"group2:  {group2}")
        cmd = f"bedops --element-of {condiction} {path1} {path2}"
        result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
        result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                 columns=["sseqid", "sstart", "send"])
        result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
        if index == 0:  # for the plus loop
            comparison_dict[name1] = [["plus", result_df.shape[0]], TP_group.shape[0]]
        else:  # for the minus loop
            comparison_dict[name1].append(["minus", result_df.shape[0], TP_group.shape[0]])

AttributeError: 'str' object has no attribute 'shape'